**Actividad - Autoencoder para detección de fraude**

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score
)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

if '__file__' in globals():
    ruta_base = os.path.dirname(os.path.abspath(__file__))
else:
    ruta_base = os.getcwd()

ruta_datos = os.path.join(ruta_base, "..", "data", "application_train.csv")
datos = pd.read_csv(ruta_datos)

filas, columnas = datos.shape
tasa_incumplimiento = datos["TARGET"].mean() * 100
casos_normales = (datos["TARGET"] == 0).sum()
casos_incumplidos = (datos["TARGET"] == 1).sum()

print(f"\nArchivo cargado correctamente: {filas} filas y {columnas} columnas")
print(f"Tasa de incumplimiento: {tasa_incumplimiento:.2f}%")
print(f"Casos normales: {casos_normales} | Casos incumplidos: {casos_incumplidos}")




Archivo cargado correctamente: 307511 filas y 122 columnas
Tasa de incumplimiento: 8.07%
Casos normales: 282686 | Casos incumplidos: 24825


In [4]:
# Selección de variables y creación de nuevas características

# Variables de tiempo
datos['EDAD'] = -datos['DAYS_BIRTH'] / 365
datos['AÑOS_EMPLEADO'] = -datos['DAYS_EMPLOYED'] / 365
datos['AÑOS_EMPLEADO'] = datos['AÑOS_EMPLEADO'].replace(1000.67, np.nan)
datos['DIAS_CAMBIO_TELEFONO'] = -datos['DAYS_LAST_PHONE_CHANGE']
datos['DIAS_PUBLICACION_ID'] = -datos['DAYS_ID_PUBLISH']

# Ratios financieros
datos['RATIO_CREDITO_INGRESO'] = datos['AMT_CREDIT'] / datos['AMT_INCOME_TOTAL']
datos['RATIO_ANUALIDAD_INGRESO'] = datos['AMT_ANNUITY'] / datos['AMT_INCOME_TOTAL']
datos['RATIO_ANUALIDAD_CREDITO'] = datos['AMT_ANNUITY'] / datos['AMT_CREDIT']
datos['RATIO_BIENES_CREDITO'] = datos['AMT_GOODS_PRICE'] / datos['AMT_CREDIT']
datos['INGRESO_PER_CAPITA'] = datos['AMT_INCOME_TOTAL'] / (datos['CNT_FAM_MEMBERS'] + 1)
datos['CREDITO_PER_CAPITA'] = datos['AMT_CREDIT'] / (datos['CNT_FAM_MEMBERS'] + 1)

# Detección de inconsistencias
datos['INCONS_SCORE_INGRESO'] = 0
mask1 = (datos['EXT_SOURCE_2'] < 0.3) & (datos['AMT_INCOME_TOTAL'] > datos['AMT_INCOME_TOTAL'].quantile(0.75))
datos.loc[mask1, 'INCONS_SCORE_INGRESO'] = 1

datos['INCONS_FAMILIA_INGRESO'] = 0
mask2 = (datos['CNT_FAM_MEMBERS'] >= 4) & (datos['AMT_INCOME_TOTAL'] < datos['AMT_INCOME_TOTAL'].quantile(0.25))
datos.loc[mask2, 'INCONS_FAMILIA_INGRESO'] = 1

columnas_docs = [col for col in datos.columns if 'FLAG_DOCUMENT' in col]
datos['TOTAL_DOCUMENTOS'] = datos[columnas_docs].sum(axis=1)
datos['INCONS_DOCS_CREDITO'] = 0
mask3 = (datos['TOTAL_DOCUMENTOS'] <= 2) & (datos['AMT_CREDIT'] > datos['AMT_CREDIT'].quantile(0.75))
datos.loc[mask3, 'INCONS_DOCS_CREDITO'] = 1

datos['TOTAL_INCONSISTENCIAS'] = (datos['INCONS_SCORE_INGRESO'] + 
                                   datos['INCONS_FAMILIA_INGRESO'] + 
                                   datos['INCONS_DOCS_CREDITO'])

# Variables de círculo social
datos['DEFAULTS_SOCIALES'] = (datos['DEF_30_CNT_SOCIAL_CIRCLE'].fillna(0) + 
                               datos['DEF_60_CNT_SOCIAL_CIRCLE'].fillna(0))
datos['OBSERVACIONES_SOCIALES'] = (datos['OBS_30_CNT_SOCIAL_CIRCLE'].fillna(0) + 
                                    datos['OBS_60_CNT_SOCIAL_CIRCLE'].fillna(0))
datos['TASA_DEFAULT_SOCIAL'] = datos['DEFAULTS_SOCIALES'] / (datos['OBSERVACIONES_SOCIALES'] + 1)
datos['TIENE_CIRCULO_RIESGOSO'] = (datos['DEFAULTS_SOCIALES'] > 0).astype(int)

# Variables de estabilidad
datos['TIENE_AUTO'] = (~datos['OWN_CAR_AGE'].isna()).astype(int)
datos['TIENE_PROPIEDAD'] = (datos['FLAG_OWN_REALTY'] == 'Y').astype(int)
datos['CAMBIO_TELEFONO_RECIENTE'] = (datos['DIAS_CAMBIO_TELEFONO'] < 180).astype(int)
datos['PUBLICACION_ID_RECIENTE'] = (datos['DIAS_PUBLICACION_ID'] < 365).astype(int)

datos['SCORE_ESTABILIDAD'] = (datos['TIENE_AUTO'] + datos['TIENE_PROPIEDAD'] + 
                               (1 - datos['CAMBIO_TELEFONO_RECIENTE']) + 
                               (1 - datos['PUBLICACION_ID_RECIENTE']))

# Interacciones y flags de riesgo
datos['RATIO_EDAD_EMPLEO'] = datos['EDAD'] / (datos['AÑOS_EMPLEADO'] + 1)
datos['SOBREENDEUDAMIENTO'] = (datos['RATIO_CREDITO_INGRESO'] > 8).astype(int)
datos['JOVEN_CREDITO_ALTO'] = ((datos['EDAD'] < 25) & (datos['AMT_CREDIT'] > 500000)).astype(int)
datos['EMPLEO_NUEVO_CREDITO_ALTO'] = ((datos['AÑOS_EMPLEADO'] < 0.5) & (datos['AMT_CREDIT'] > 300000)).astype(int)

columnas_bureau = [col for col in datos.columns if 'AMT_REQ_CREDIT_BUREAU' in col]
datos['TOTAL_CONSULTAS_BUREAU'] = datos[columnas_bureau].sum(axis=1)
datos['CONSULTAS_EXCESIVAS'] = (datos['TOTAL_CONSULTAS_BUREAU'] > 5).astype(int)

# Lista de variables seleccionadas
variables_seleccionadas = [
    'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
    'RATIO_CREDITO_INGRESO', 'RATIO_ANUALIDAD_INGRESO', 'RATIO_ANUALIDAD_CREDITO',
    'INGRESO_PER_CAPITA', 'CREDITO_PER_CAPITA',
    'EXT_SOURCE_2', 'EXT_SOURCE_3',
    'EDAD', 'AÑOS_EMPLEADO', 'DAYS_REGISTRATION', 'RATIO_EDAD_EMPLEO',
    'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'TASA_DEFAULT_SOCIAL', 'TIENE_CIRCULO_RIESGOSO',
    'REGION_RATING_CLIENT', 'REG_CITY_NOT_WORK_CITY',
    'TOTAL_DOCUMENTOS', 'SCORE_ESTABILIDAD',
    'TOTAL_INCONSISTENCIAS', 'INCONS_SCORE_INGRESO', 
    'INCONS_FAMILIA_INGRESO', 'INCONS_DOCS_CREDITO',
    'SOBREENDEUDAMIENTO', 'JOVEN_CREDITO_ALTO', 'EMPLEO_NUEVO_CREDITO_ALTO', 
    'CONSULTAS_EXCESIVAS', 'CAMBIO_TELEFONO_RECIENTE'
]

print(f"\nVariables creadas: {len(variables_seleccionadas)}")
print(f"Inconsistencias detectadas: {datos['TOTAL_INCONSISTENCIAS'].sum()}")


Variables creadas: 31
Inconsistencias detectadas: 89958


In [5]:
# PREPARACION DE DATOS

# Crear dataset de trabajo
datos_trabajo = datos[variables_seleccionadas + ['TARGET', 'SK_ID_CURR']].copy()

# Imputar valores nulos
for col in variables_seleccionadas:
    if datos_trabajo[col].isnull().sum() > 0:
        if col in ['EXT_SOURCE_2', 'EXT_SOURCE_3']:
            datos_trabajo[col].fillna(datos_trabajo[col].median(), inplace=True)
        else:
            datos_trabajo[col].fillna(0, inplace=True)

# Separar datos normales para entrenar el autoencoder
datos_normales = datos_trabajo[datos_trabajo['TARGET'] == 0].copy()
X_normales = datos_normales[variables_seleccionadas].values
X_todos = datos_trabajo[variables_seleccionadas].values
y_todos = datos_trabajo['TARGET'].values

# Dividir en entrenamiento y validacion
X_entrenamiento, X_validacion = train_test_split(X_normales, test_size=0.2, random_state=42)

# Escalar datos
escalador = RobustScaler()
X_entrenamiento_escalado = escalador.fit_transform(X_entrenamiento)
X_validacion_escalado = escalador.transform(X_validacion)
X_todos_escalado = escalador.transform(X_todos)

print(f"\nDatos preparados:")
print(f"Entrenamiento: {X_entrenamiento.shape}")
print(f"Validacion: {X_validacion.shape}")
print(f"Dataset completo: {X_todos.shape}")


Datos preparados:
Entrenamiento: (226148, 31)
Validacion: (56538, 31)
Dataset completo: (307511, 31)


In [6]:
# CONSTRUCCION Y ENTRENAMIENTO DEL AUTOENCODER

# Definir arquitectura del autoencoder
dimension_entrada = X_entrenamiento_escalado.shape[1]

# Capa de entrada
entrada = Input(shape=(dimension_entrada,))

# Encoder: comprime la informacion
x = Dense(128, activation='relu')(entrada)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(32, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Capa latente (representacion comprimida)
codigo = Dense(16, activation='relu')(x)

# Decoder: reconstruye la informacion
x = Dense(32, activation='relu')(codigo)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)

# Capa de salida
salida = Dense(dimension_entrada, activation='linear')(x)

# Crear y compilar el modelo
autoencoder = Model(entrada, salida)
autoencoder.compile(optimizer='adam', loss='mae', metrics=['mse'])

print(f"\nModelo creado con {dimension_entrada} variables de entrada")
autoencoder.summary()

# Configurar callbacks para el entrenamiento
parada_temprana = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reducir_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, verbose=1)

# Entrenar el modelo
print("\nEntrenando autoencoder...")
historial = autoencoder.fit(
    X_entrenamiento_escalado, X_entrenamiento_escalado,
    epochs=150,
    batch_size=256,
    validation_data=(X_validacion_escalado, X_validacion_escalado),
    callbacks=[parada_temprana, reducir_lr],
    verbose=1
)

print(f"\nEntrenamiento completado en {len(historial.history['loss'])} epocas")
print(f"Loss final entrenamiento: {historial.history['loss'][-1]:.4f}")
print(f"Loss final validacion: {historial.history['val_loss'][-1]:.4f}")


Modelo creado con 31 variables de entrada


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 31)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 31)             │         3,999 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,727 (123.93 KB)

 Trainable params: 30,831 (120.43 KB)

 Non-trainable params: 896 (3.50 KB)


Entrenando autoencoder...
Epoch 1/150
884/884 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 1.0915 - mse: 78.0307 - val_loss: 0.3474 - val_mse: 1.3257 - learning_rate: 0.0010
Epoch 2/150
884/884 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.4019 - mse: 1.7693 - val_loss: 0.3317 - val_mse: 1.1999 - learning_rate: 0.0010
Epoch 3/150
884/884 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.3725 - mse: 1.2415 - val_loss: 0.2688 - val_mse: 0.3020 - learning_rate: 0.0010
Epoch 4/150
884/884 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.3484 - mse: 1.0631 - val_loss: 0.2617 - val_mse: 0.3571 - learning_rate: 0.0010
Epoch 5/150
884/884 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.3370 - mse: 1.0091 - val_loss: 0.2415 - val_mse: 0.2598 - learning_rate: 0.0010
Epoch 6/150
884/884 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.3361 - mse: 0.9920 - val_loss: 0.2240 - val_mse: 0.2108 - learning_rate: 0.0010
Epoch 7/150
884/884 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.3261 - mse: 0.8848 - val_loss: 0.2217 - val_mse: 0.